In [ ]:
%matplotlib inline

# Torch.nn
by Jeremy Howard, `fast.ai <https://www.fast.ai>`    

硬件：Station  
软件：fusimeng/ai.pytorch:v5


Pytorch提供了torch.nn、torch.optim、Dataset和DataLoader这些设计优雅的模块和类以帮助使用者创建和训练神经网络。 为了最大化利用这些模块和类的功能，并使用它们做出适用于你所研究问题的模型，你需要真正理解他们是如何工作的。 为了做到这一点，我们首先基于MNIST数据集训练一个没有任何特征的简单神经网络。 最开始我们只会用到PyTorch中最基本的tensor功能，然后我们将会逐渐的从torch.nn，torch.optim，Dataset，DataLoader中选择一个特征加入到模型中，来展示新加入的特征会对模型产生什么样的效果，以及它是如何使模型变得更简洁或更灵活。

## 一、MNIST data setup

我们将使[pathlib](https://docs.python.org/zh-cn/3/library/pathlib.html) 来处理路径（Python 3标准库的一部分），并将使用请求下载数据集 。我们只会在使用模块时导入模块，因此您可以准确地看到每个模块的使用情况。


In [3]:
from pathlib import Path
import requests

DATA_PATH = Path("Data")
PATH = DATA_PATH / "mnist"

PATH.mkdir(parents=True, exist_ok=True)

URL = "http://deeplearning.net/data/mnist/"
FILENAME = "mnist.pkl.gz"

if not (PATH / FILENAME).exists():
        content = requests.get(URL + FILENAME).content
        (PATH / FILENAME).open("wb").write(content)

该数据集采用numpy数组格式，并使用pickle存储，pickle是一种特定于python的格式，用于序列化数据。

In [4]:
import pickle
import gzip

with gzip.open((PATH / FILENAME).as_posix(), "rb") as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")

每个图像为28 x 28，并存储为长度为784（= 28x28）的扁平行。我们来看一个; 我们需要先将其重塑为2d。

In [5]:
from matplotlib import pyplot
import numpy as np

pyplot.imshow(x_train[0].reshape((28, 28)), cmap="gray")
print(x_train.shape)

(50000, 784)


PyTorch使用torch.tensor而不是numpy数组，因此我们需要转换数据。

In [6]:
import torch

x_train, y_train, x_valid, y_valid = map(
    torch.tensor, (x_train, y_train, x_valid, y_valid)
)
n, c = x_train.shape
x_train, x_train.shape, y_train.min(), y_train.max()
print(x_train, y_train)
print(x_train.shape)
print(y_train.min(), y_train.max())

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]) tensor([5, 0, 4,  ..., 8, 4, 8])
torch.Size([50000, 784])
tensor(0) tensor(9)


## 二、从头开始的神经网络（没有torch.nn）
让我们首先使用PyTorch张量操作创建一个模型。我们假设您已经熟悉神经网络的基础知识。（如果你不是，你可以在[course.fast.ai](https://course.fast.ai/)学习它们）。

PyTorch提供了创建随机或零填充张量的方法，我们将使用这些方法为简单的线性模型创建权重和偏差。这些只是常规张量，有一个非常特别的补充：我们告诉PyTorch它们需要一个渐变。这会导致PyTorch记录在张量上完成的所有操作，以便它可以在反向传播过程中自动计算梯度！

对于权重，我们requires_grad 在初始化之后设置，因为我们不希望梯度中包含该步骤。（请注意，PyTorch _中的跟踪表示操作是就地执行的。）

In [7]:
import math

weights = torch.randn(784, 10) / math.sqrt(784)
weights.requires_grad_()
bias = torch.zeros(10, requires_grad=True)

由于PyTorch能够自动计算渐变，我们可以使用任何标准的Python函数（或可调用对象）作为模型！因此，让我们编写一个简单的矩阵乘法和广播加法来创建一个简单的线性模型。我们还需要一个激活函数，所以我们将编写log_softmax并使用它。记住：虽然PyTorch提供了许多预先编写的丢失函数，激活函数等，但您可以使用普通的python轻松编写自己的函数。PyTorch甚至可以自动为您的功能创建快速GPU或矢量化CPU代码。

In [8]:
def log_softmax(x):
    return x - x.exp().sum(-1).log().unsqueeze(-1)

def model(xb):
    return log_softmax(xb @ weights + bias)

在上面，@代表点积运算。我们将在一批数据上调用我们的函数（在这种情况下，64个图像）。这是一个前进传球。请注意，我们的预测在这个阶段不会比随机更好，因为我们从随机权重开始。

In [9]:
bs = 64  # batch size

xb = x_train[0:bs]  # a mini-batch from x
preds = model(xb)  # predictions
preds[0], preds.shape
print(preds[0], preds.shape)

tensor([-2.6939, -2.7041, -1.8446, -2.8252, -2.3655, -2.1982, -2.9234, -2.0235,
        -2.4049, -1.7901], grad_fn=<SelectBackward>) torch.Size([64, 10])


如您所见，preds张量不仅包含张量值，还包含渐变函数。我们稍后会用它来做backprop。

让我们实现负对数似然用作损失函数（同样，我们可以使用标准Python）：

In [10]:
def nll(input, target):
    return -input[range(target.shape[0]), target].mean()

loss_func = nll

让我们用我们的随机模型检查我们的损失，这样我们就可以看到我们是否在后面的backprop传递后改进了。

In [11]:
yb = y_train[0:bs]
print(loss_func(preds, yb))

tensor(2.4342, grad_fn=<NegBackward>)


我们还实现了一个计算模型精度的函数。对于每个预测，如果具有最大值的索引与目标值匹配，则预测是正确的。

In [12]:
def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

让我们检查一下随机模型的准确性，这样我们就可以看出随着损失的改善我们的准确度是否会提高。

In [13]:
print(accuracy(preds, yb))

tensor(0.1250)


我们现在可以运行一个训练循环。对于每次迭代，我们将：

* 选择一小批数据（大小bs）
* 使用该模型进行预测
* 计算损失
* loss.backward()在这种情况下，更新模型的渐变，weights 和bias。
  
我们现在使用这些渐变来更新权重和偏差。我们在torch.no_grad()上下文管理器中执行此操作，因为我们不希望记录这些操作以用于下一次计算渐变。您可以在此处详细了解PyTorch的Autograd如何记录操作 。

然后我们将梯度设置为零，以便我们为下一个循环做好准备。否则，我们的渐变将记录已发生的所有操作的运行记录（即将渐变loss.backward() 添加到已存储的任何内容，而不是替换它们）。   
小贴士

您可以使用标准python调试器对PyTorch代码进行单步调试，从而在每一步检查不同的变量值。取消下面的set_trace()来尝试该功能。

In [14]:
from IPython.core.debugger import set_trace

lr = 0.5  # learning rate
epochs = 2  # how many epochs to train for

for epoch in range(epochs):
    for i in range((n - 1) // bs + 1):
        #         set_trace()
        start_i = i * bs
        end_i = start_i + bs
        xb = x_train[start_i:end_i]
        yb = y_train[start_i:end_i]
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        with torch.no_grad():
            weights -= weights.grad * lr
            bias -= bias.grad * lr
            weights.grad.zero_()
            bias.grad.zero_()

就是这样：我们创建并训练了一个最小的神经网络（在这种情况下，逻辑回归，因为我们没有隐藏的层）完全从头开始！

让我们检查损失和准确性，并将它们与我们之前得到的结果进行比较。我们预计损失会减少，准确性会增加，而且他们有。

In [15]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))

tensor(0.0805, grad_fn=<NegBackward>) tensor(1.)


## 三、使用torch.nn.functional
我们现在将重构我们的代码，以便它像以前一样做，只有我们才会开始利用PyTorch的nn类来使它更简洁和灵活。在这里的每一步，我们应该使我们的代码中的一个或多个：更短，更易理解和/或更灵活。

第一个也是最简单的步骤是通过将手写的激活和丢失函数替换为来自torch.nn.functional （通常F按惯例导入到命名空间中）的函数来缩短代码。该模块包含torch.nn库中的所有函数（而库的其他部分包含类）。除了各种各样的丢失和激活功能外，您还可以在这里找到一些用于创建神经网络的便捷功能，例如池功能。（还有用于进行卷积，线性层等的函数，但正如我们将看到的，通常可以使用库的其他部分更好地处理这些函数。）

如果您使用负对数似然丢失和记录softmax激活，那么Pytorch提供了一个F.cross_entropy将两者结合起来的功能。所以我们甚至可以从我们的模型中删除激活函数。

In [16]:
import torch.nn.functional as F

loss_func = F.cross_entropy

def model(xb):
    return xb @ weights + bias

请注意，我们不再调用log_softmax该model函数。让我们确认我们的损失和准确性与以前相同：

In [17]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))

tensor(0.0805, grad_fn=<NllLossBackward>) tensor(1.)


## 四、使用nn.Module重构
接下来，我们将使用nn.Module和nn.Parameter，以获得更清晰，更简洁的训练循环。我们子类nn.Module（它本身是一个类，能够跟踪状态）。在这种情况下，我们想要创建一个包含前向步骤的权重，偏差和方法的类。 我们将使用nn.Module许多属性和方法（例如.parameters()和.zero_grad()）。

In [18]:
from torch import nn

class Mnist_Logistic(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(784, 10) / math.sqrt(784))
        self.bias = nn.Parameter(torch.zeros(10))

    def forward(self, xb):
        return xb @ self.weights + self.bias

由于我们现在使用的是对象而不是仅使用函数，因此我们首先必须实例化我们的模型：

In [19]:
model = Mnist_Logistic()

现在我们可以像以前一样计算损失。请注意， nn.Module对象被用作函数（即它们是 可调用的），但在幕后，Pytorch将forward 自动调用我们的方法。

In [20]:
print(loss_func(model(xb), yb))

tensor(2.4095, grad_fn=<NllLossBackward>)


以前，对于我们的训练循环，我们必须按名称更新每个参数的值，并分别手动将每个参数的梯度归零，如下所示：
```
with torch.no_grad():
    weights -= weights.grad * lr
    bias -= bias.grad * lr
    weights.grad.zero_()
    bias.grad.zero_()
```
现在我们可以利用model.parameters（）和model.zero_grad（）（它们都由PyTorch定义nn.Module）来使这些步骤更简洁，更不容易忘记我们的一些参数，特别是如果我们有一个更复杂的模型：
```
with torch.no_grad():
    for p in model.parameters(): p -= p.grad * lr
    model.zero_grad()
```
我们将把我们的小训练循环包装在一个fit函数中，以便稍后再运行它。



In [21]:
def fit():
    for epoch in range(epochs):
        for i in range((n - 1) // bs + 1):
            start_i = i * bs
            end_i = start_i + bs
            xb = x_train[start_i:end_i]
            yb = y_train[start_i:end_i]
            pred = model(xb)
            loss = loss_func(pred, yb)

            loss.backward()
            with torch.no_grad():
                for p in model.parameters():
                    p -= p.grad * lr
                model.zero_grad()

fit()

让我们仔细检查一下我们的损失是否已经下降：

In [22]:
print(loss_func(model(xb), yb))

tensor(0.0822, grad_fn=<NllLossBackward>)


## 五、使用nn.Linear重构
我们继续重构我们的代码。我们将使用Pytorch类nn.Linear而不是手动定义和初始化self.weights和self.bias计算，而是使用 线性层，它为我们做了所有这些。Pytorch有许多类型的预定义层，可以大大简化我们的代码，并且通常也使它更快。xb  @ self.weights + self.bias
```python
class Mnist_Logistic(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin = nn.Linear(784, 10)

    def forward(self, xb):
        return self.lin(xb)
```
我们实例化我们的模型并以与以前相同的方式计算损失：
```
model = Mnist_Logistic()
print(loss_func(model(xb), yb))
```

In [23]:
class Mnist_Logistic(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin = nn.Linear(784, 10)

    def forward(self, xb):
        return self.lin(xb)

We instantiate our model and calculate the loss in the same way as before:



In [24]:
model = Mnist_Logistic()
print(loss_func(model(xb), yb))

tensor(2.2493, grad_fn=<NllLossBackward>)


We are still able to use our same ``fit`` method as before.



In [25]:
fit()

print(loss_func(model(xb), yb))

tensor(0.0820, grad_fn=<NllLossBackward>)


### 六、重构使用optim
Pytorch还有一个包含各种优化算法的包torch.optim。我们可以使用step优化器中的方法来执行前进步骤，而不是手动更新每个参数。

这将让我们取代之前的手动编码优化步骤：
```
with torch.no_grad():
    for p in model.parameters(): p -= p.grad * lr
    model.zero_grad()
```
而只是使用：
```
opt.step()
opt.zero_grad()
```
（optim.zero_grad()将渐变重置为0，我们需要在计算下一个小批量的渐变之前调用它。）

from torch import optim
我们将定义一个小函数来创建我们的模型和优化器，以便将来可以重用它。

In [26]:
from torch import optim

We'll define a little function to create our model and optimizer so we
can reuse it in the future.



In [27]:
def get_model():
    model = Mnist_Logistic()
    return model, optim.SGD(model.parameters(), lr=lr)

model, opt = get_model()
print(loss_func(model(xb), yb))

for epoch in range(epochs):
    for i in range((n - 1) // bs + 1):
        start_i = i * bs
        end_i = start_i + bs
        xb = x_train[start_i:end_i]
        yb = y_train[start_i:end_i]
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

print(loss_func(model(xb), yb))

tensor(2.3471, grad_fn=<NllLossBackward>)
tensor(0.0806, grad_fn=<NllLossBackward>)


## 七、使用数据集重构
PyTorch有一个抽象的Dataset类。数据集可以是任何具有__len__函数（由Python的标准len函数调用）和__getitem__函数作为索引的方式。 本教程 将介绍创建自定义FacialLandmarkDataset类作为子类的一个很好的示例Dataset。

PyTorch的TensorDataset 是一个包含张量的数据集。通过定义索引的长度和方式，这也为我们提供了一种沿张量的第一维迭代，索引和切片的方法。这将使我们更容易在我们训练的同一行中访问独立变量和因变量。


In [28]:
from torch.utils.data import TensorDataset


二者x_train并y_train可以在一个单一的组合TensorDataset，这将是容易遍历和切片。



In [29]:
train_ds = TensorDataset(x_train, y_train)


以前，我们必须分别迭代x和y值的小批量：

xb = x_train[start_i:end_i]
yb = y_train[start_i:end_i]
现在，我们可以一起完成这两个步骤：

xb,yb = train_ds[i*bs : i*bs+bs]


In [30]:
model, opt = get_model()

for epoch in range(epochs):
    for i in range((n - 1) // bs + 1):
        xb, yb = train_ds[i * bs: i * bs + bs]
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

print(loss_func(model(xb), yb))

tensor(0.0813, grad_fn=<NllLossBackward>)


## 八、使用DataLoader重构
Pytorch DataLoader负责管理批次。您可以创建一个DataLoader从任何Dataset。DataLoader使批量迭代变得更容易。DataLoader 不是必须使用，而是自动为我们提供每个小批量。train_ds[i*bs : i*bs+bs]



In [31]:
from torch.utils.data import DataLoader

train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=bs)

以前，我们的循环遍历批次（xb，yb），如下所示：
```
for i in range((n-1)//bs + 1):
    xb,yb = train_ds[i*bs : i*bs+bs]
    pred = model(xb)
```
现在，我们的循环更清晰，因为（xb，yb）是从数据加载器自动加载的：
```
for xb,yb in train_dl:
    pred = model(xb)
```

In [32]:
model, opt = get_model()

for epoch in range(epochs):
    for xb, yb in train_dl:
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

print(loss_func(model(xb), yb))

tensor(0.0827, grad_fn=<NllLossBackward>)


由于Pytorch的nn.Module，nn.Parameter，Dataset，和DataLoader，我们的训练循环现显着更小，更容易理解。现在让我们尝试添加在实践中创建有效模型所需的基本功能。

## 九、添加验证
在第1部分中，我们只是尝试设置合理的训练循环以用于我们的训练数据。实际上，您总是应该有一个验证集，以确定您是否过度拟合。

改变训练数据 对于防止批次与过度拟合之间的相关性非常 重要。另一方面，无论我们是否对验证集进行洗牌，验证损失都是相同的。由于改组需要额外的时间，因此对验证数据进行洗牌是没有意义的。

我们将使用批量大小作为验证集，其大小是训练集的两倍。这是因为验证集不需要反向传播，因此占用更少的内存（它不需要存储渐变）。我们利用这一点来使用更大的批量大小并更快地计算损失。



In [33]:
train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)

valid_ds = TensorDataset(x_valid, y_valid)
valid_dl = DataLoader(valid_ds, batch_size=bs * 2)

我们将在每个时代结束时计算并打印验证损失。

（请注意，我们总是model.train()在训练之前和model.eval() 推理之前调用，因为这些是由层使用的，例如nn.BatchNorm2d 并nn.Dropout确保这些不同阶段的适当行为。）

In [34]:
model, opt = get_model()

for epoch in range(epochs):
    model.train()
    for xb, yb in train_dl:
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

    model.eval()
    with torch.no_grad():
        valid_loss = sum(loss_func(model(xb), yb) for xb, yb in valid_dl)

    print(epoch, valid_loss / len(valid_dl))

0 tensor(0.3207)
1 tensor(0.2862)


## 十、创建fit（）和get_data（）
我们现在将对自己进行一些重构。由于我们经历了两次计算训练集和验证集损失的类似过程，因此我们将其转换为自己的函数loss_batch，该函数计算一个批次的损失。

我们为训练集传递一个优化器，并使用它来执行backprop。对于验证集，我们不传递优化器，因此该方法不执行backprop。

In [35]:
def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)

fit 运行必要的操作来训练我们的模型并计算每个时期的训练和验证损失。

In [36]:
import numpy as np

def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_dl:
            loss_batch(model, loss_func, xb, yb, opt)

        model.eval()
        with torch.no_grad():
            losses, nums = zip(
                *[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl]
            )
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)

        print(epoch, val_loss)

``get_data`` returns dataloaders for the training and validation sets.



In [37]:
def get_data(train_ds, valid_ds, bs):
    return (
        DataLoader(train_ds, batch_size=bs, shuffle=True),
        DataLoader(valid_ds, batch_size=bs * 2),
    )

Now, our whole process of obtaining the data loaders and fitting the
model can be run in 3 lines of code:



In [38]:
train_dl, valid_dl = get_data(train_ds, valid_ds, bs)
model, opt = get_model()
fit(epochs, model, loss_func, opt, train_dl, valid_dl)

0 0.4850252965450287
1 0.30622428317070005


You can use these basic 3 lines of code to train a wide variety of models.
Let's see if we can use them to train a convolutional neural network (CNN)!

## 十一、Switch to CNN

我们现在要用三个卷积层构建我们的神经网络。因为上一节中没有任何函数假定模型形式的任何内容，我们将能够使用它们来训练CNN而无需任何修改。

我们将使用Pytorch的预定义 Conv2d类作为卷积层。我们定义了一个带有3个卷积层的CNN。每个卷积后面都有一个ReLU。最后，我们执行平均合并。（注意这view是PyTorch的numpy版本 reshape）


In [39]:
class Mnist_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1)

    def forward(self, xb):
        xb = xb.view(-1, 1, 28, 28)
        xb = F.relu(self.conv1(xb))
        xb = F.relu(self.conv2(xb))
        xb = F.relu(self.conv3(xb))
        xb = F.avg_pool2d(xb, 4)
        return xb.view(-1, xb.size(1))

lr = 0.1

`Momentum <https://cs231n.github.io/neural-networks-3/#sgd>`_ is a variation on
stochastic gradient descent that takes previous updates into account as well
and generally leads to faster training.



In [40]:
model = Mnist_CNN()
opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

fit(epochs, model, loss_func, opt, train_dl, valid_dl)

0 0.44883635759353635
1 0.2483194197177887


## 十二、nn.Sequential
torch.nn有另一个方便的类我们可以使用简单的代码： 顺序。甲Sequential对象运行的每个包含在其内的模块的，以顺序的方式。这是编写神经网络的一种更简单的方法。

要利用这一点，我们需要能够轻松地从给定函数定义 自定义图层。例如，PyTorch没有视图层，我们需要为我们的网络创建一个。Lambda 将创建一个层，我们可以在定义网络时使用 Sequential。



In [41]:
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x):
        return self.func(x)


def preprocess(x):
    return x.view(-1, 1, 28, 28)

The model created with ``Sequential`` is simply:



In [42]:
model = nn.Sequential(
    Lambda(preprocess),
    nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.AvgPool2d(4),
    Lambda(lambda x: x.view(x.size(0), -1)),
)

opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

fit(epochs, model, loss_func, opt, train_dl, valid_dl)

0 0.4737539008140564
1 0.2766986144065857


## 十三、包装DataLoader
我们的CNN相当简洁，但它只适用于MNIST，因为：
它假设输入是28 * 28长矢量
它假设最终CNN网格大小为4 * 4（因为这是平均值）
汇集我们使用的内核大小）

让我们摆脱这两个假设，因此我们的模型适用于任何二维单通道图像。首先，我们可以删除初始Lambda层，但将数据预处理移动到生成器中：



In [43]:
def preprocess(x, y):
    return x.view(-1, 1, 28, 28), y


class WrappedDataLoader:
    def __init__(self, dl, func):
        self.dl = dl
        self.func = func

    def __len__(self):
        return len(self.dl)

    def __iter__(self):
        batches = iter(self.dl)
        for b in batches:
            yield (self.func(*b))

train_dl, valid_dl = get_data(train_ds, valid_ds, bs)
train_dl = WrappedDataLoader(train_dl, preprocess)
valid_dl = WrappedDataLoader(valid_dl, preprocess)

Next, we can replace ``nn.AvgPool2d`` with ``nn.AdaptiveAvgPool2d``, which
allows us to define the size of the *output* tensor we want, rather than
the *input* tensor we have. As a result, our model will work with any
size input.



In [44]:
model = nn.Sequential(
    nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.AdaptiveAvgPool2d(1),
    Lambda(lambda x: x.view(x.size(0), -1)),
)

opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

Let's try it out:



In [45]:
fit(epochs, model, loss_func, opt, train_dl, valid_dl)

0 0.33956847071647644
1 0.2703994888305664


十四、Using your GPU
---------------

If you're lucky enough to have access to a CUDA-capable GPU (you can
rent one for about $0.50/hour from most cloud providers) you can
use it to speed up your code. First check that your GPU is working in
Pytorch:



In [46]:
print(torch.cuda.is_available())

True


And then create a device object for it:



In [47]:
dev = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")

Let's update ``preprocess`` to move batches to the GPU:



In [48]:
def preprocess(x, y):
    return x.view(-1, 1, 28, 28).to(dev), y.to(dev)


train_dl, valid_dl = get_data(train_ds, valid_ds, bs)
train_dl = WrappedDataLoader(train_dl, preprocess)
valid_dl = WrappedDataLoader(valid_dl, preprocess)

Finally, we can move our model to the GPU.



In [49]:
model.to(dev)
opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

You should find it runs faster now:



In [50]:
fit(epochs, model, loss_func, opt, train_dl, valid_dl)

0 0.1940934679031372
1 0.1676538721561432


## 十六、结束思想
我们现在有一个通用数据管道和训练循环，您可以使用它来使用Pytorch训练多种类型的模型。要了解模型现在的简单培训，请查看mnist_sample示例笔记本。

当然，您需要添加许多内容，例如数据增强，超参数调整，监控培训，转移学习等等。这些功能在fastai库中提供，该库使用本教程中显示的相同设计方法开发，为希望进一步采用其模型的从业者提供了自然的下一步。

我们承诺在本教程的开始，我们会通过例如每个解释 torch.nn，torch.optim，Dataset，和DataLoader。那么让我们总结一下我们所看到的：

* torch.nn
    * Module：创建一个callable，其行为类似于函数，但也可以包含状态（例如神经网络层权重）。它知道Parameter它包含什么，并且可以将所有渐变归零，循环通过它们以进行重量更新等。
    * Parameter：张量的包装器，告诉Module它在backprop期间它有需要更新的权重。仅更新具有requires_grad属性集的张量
    * functional：一个模块（通常F按惯例导入到命名空间中），它包含激活函数，损失函数等，以及层的非有状态版本，如卷积层和线性层。
* torch.optim：包含优化程序，例如SGD，更新Parameter后退步骤中的权重
* Dataset：带有a __len__和a 的对象的抽象接口__getitem__，包括用Pytorch提供的类，如TensorDataset
* DataLoader：获取任何Dataset并创建一个返回批量数据的迭代器。